# Ungridded In Situ: Global timeseries

## Import libraries

Switch warnings off for better readability.

In [ ]:
import warnings

import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import download, plot

warnings.filterwarnings("ignore")

## Define request

In [ ]:
collection_id = "insitu-observations-gruan-reference-network"
request = {
    "format": "csv-lev.zip",
    "variable": "air_temperature",
}
start = "2018-01"
stop = "2020-01"

## Generate requests

In [ ]:
requests = download.update_request_date(request, start=start, stop=stop)

## Define transform function:

1. Convert timestamp to datetime
1. Rename variables using station names
1. Compute chunk mean and std

In [ ]:
def transform_func(ds):
    ds["datetime"] = pd.to_datetime(ds["report_timestamp"])
    datetime = pd.to_datetime(ds["report_timestamp"])
    time = pd.to_datetime(f"{datetime.mean().year}-{datetime.mean().month}")
    dataarrays = []
    for station, dataset in ds.groupby("station_name"):
        da = ds["air_temperature"]
        for method in ("mean", "std"):
            reduced = (
                getattr(da, method)()
                .rename(station)
                .expand_dims(time=[time], reduction=[method])
            )
            dataarrays.append(reduced)
    return xr.merge(dataarrays)

## Compute monthly mean and std timeseries

In [ ]:
ds = download.download_and_transform(
    collection_id,
    requests,
    chunks={"year": 1, "month": 1},
    transform_func=transform_func,
)

## Plot and save figure

In [ ]:
filename = "_".join(["global-timeseries", collection_id])
title = filename.replace("-", " ").replace("_", " ").title()
fig = plot.shaded_std(
    list(ds.data_vars),
    ds.sel(reduction="mean"),
    title=title,
)
fig.show()
fig.write_image(filename + ".png")